# LOG6308 : Tp2 - Approche contenu

- Clément Bernard (2096223)
- Ghaith Dekhili (1858454)

## Importations 

In [ ]:
#We need to put diag to 0 in citeseer to eliminate self cited papers

In [227]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import os
from sklearn.metrics.pairwise import cosine_similarity


from scipy.spatial import distance
import time
from sklearn import preprocessing
from numpy.linalg import multi_dot

In [43]:
DATA = os.path.join(os.getcwd(), 'data')
TABLE_PATH = os.path.join(DATA, 'citeseer.rtable')
ABSTRACT_PATH = os.path.join(DATA,'abstracts.csv')

## Data 


In [44]:
# Matrice d'adjacence
m = pd.read_table(TABLE_PATH, sep=' ')
# Number of articles
N = m.shape[0]

# Question 1

## Data

In [229]:
# The path where is the fold data
PATH_DATA = 'data'

# The items 
items = pd.read_csv(os.path.join(PATH_DATA, 'items.csv'), sep='|')
# User data 
u = pd.read_csv(os.path.join(PATH_DATA, 'u.csv'), sep='|')
# Votes of the user 
votes = pd.read_csv(os.path.join(PATH_DATA, 'votes.csv'), sep='|')

In [230]:
items

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [231]:
u

,id,age,gender,job,zip
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [232]:
votes

,user.id,item.id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


## Creation of sparse matrix : User-item matrix

In [233]:
# The number of users 
N_USERS = u.shape[0] +1
# The number of items 
N_ITEMS = items.shape[0] +1 

In [234]:
def create_sparse_matrix(votes) : 
    ''' Create a User-Items sparse matrix '''
    # Create NaN for each items and users 
    data = {i : [np.nan for j in range(N_USERS )] for i in range(N_ITEMS)}
    def to_convert(x, data) :
        data[x['item.id']][x['user.id']] = x['rating']
        return None 
    votes.apply(to_convert , axis = 1 , args = [data])
    return pd.DataFrame(data)

In [235]:
user_item = create_sparse_matrix(votes)

In [236]:
user_item

,0,1,2,3,4,5,6,7,8,9,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,NaN,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [237]:
# We drop the first user who has no grades
user_item= user_item.iloc[1:,1:]

In [243]:
def convert_to_binary(user_item):
    """
    returns a binary matrix by replacing votes equal or greater than 4 by 1 and 0 otherwise
    """
    user_item_binary = user_item.copy()
    # votes with 4 and 5 values become 1 and 0 otherwise 
    user_item_binary[user_item_binary < 4] = 0
    user_item_binary[user_item_binary >= 4] = 1
    return user_item_binary

In [244]:
def count_global_likes(user_item):
    """
    Counts global likes dislikes per item
    """
    #A df with number of likes and dislikes per item
    items_global_likes = pd.DataFrame(np.zeros((N_ITEMS-1, 2)), index = range(1,N_ITEMS))
    user_item_binary = convert_to_binary(user_item)
    for item in range(1,N_ITEMS):
        #Count number of 1(likes) and 0(dislikes) per item
        likes = user_item_binary[item].value_counts(normalize=False)
        #Fill with global number of likes and likes per item
        items_global_likes.loc[item,:] = likes
        #items_global_likes.loc[item,["dislikes","likes"]] = likes
    return user_item_binary, items_global_likes

In [245]:
def labelize_age(u):
    """
    labelize age based on users who are older than 25 are considered old
    rerturn a pandas Series with labelized ages
    """
    age_categorical = u[' age '].copy()
    age_categorical[u[' age '] > 25 ] = 'old'
    age_categorical[u[' age '] <= 25] = 'young'
    age_categorical = pd.DataFrame({'id ':[int(idx) for idx in u['id '].values],
                                    ' age ': age_categorical.values}, index = range(1,N_USERS))          
    return age_categorical

In [246]:
def return_dict_feature_ids(u,column, age = False):
    """
    returns a dictionary of a feature categories with their associated users ids
    """
    feature_per_ids = {}
    #A df with a feature and its associated ids users, for we you labelize ages 
    if age:
        df_feature = labelize_age(u)
    else:
        df_feature = u[['id ', column]].copy() 
    #group by feature per user_ids
    feature_per_ids = {name: group['id '].values for name,group in df_feature.groupby([column])}
    return feature_per_ids

In [247]:
def return_likes_dislikes_per_feature(u, user_item_binary, feature_per_ids):
    """
    returns likes dislikes matrices per job
    """
    #initializations
    item_likes_per_job = pd.DataFrame(np.nan, index = feature_per_ids.keys(), columns = range(1,N_ITEMS))
    item_dislikes_per_job = pd.DataFrame(np.nan, index = feature_per_ids.keys(), columns = range(1,N_ITEMS))
    #Loop over features and associated users ids
    for job, ids in feature_per_ids.items():
        item_likes_per_job.loc[job] = user_item_binary.loc[ids][user_item_binary.loc[ids]==1].count()
        item_dislikes_per_job.loc[job] = user_item_binary.loc[ids][user_item_binary.loc[ids]==0].count()
    
    return item_likes_per_job, item_dislikes_per_job

In [248]:
user_item_binary, items_global_likes = count_global_likes(user_item)
odds_likes = (items_global_likes.iloc[:,1]/items_global_likes.iloc[:,0])
proba_likes = odds_likes/(1+odds_likes)

In [249]:
#Returns a dictionary of jobs categories with their associated users ids
jobs_per_ids = return_dict_feature_ids(u, ' job ') 
#Return items likes and dislikes matrices per job
item_likes_per_job, item_dislikes_per_job = return_likes_dislikes_per_feature(u, user_item_binary, jobs_per_ids)
#calculate odds related to jobs
users_items_likes = (item_likes_per_job+1)/(((np.dot(pd.DataFrame(items_global_likes.iloc[:,1].replace(np.nan,0)+2),np.ones((1,item_likes_per_job.shape[0])))).T))
users_items_dislikes = (item_dislikes_per_job+1)/(((np.dot(pd.DataFrame(items_global_likes.iloc[:,0].replace(np.nan,0)+2),np.ones((1,item_likes_per_job.shape[0])))).T))
odds_jobs = pd.DataFrame(users_items_likes)/pd.DataFrame(users_items_dislikes)

In [250]:
#Returns a dictionary of gender categories with their associated users ids
gender_per_ids = return_dict_feature_ids(u, ' gender ')
#Return items likes and dislikes matrices per gender
item_likes_per_gender, item_dislikes_per_gender = return_likes_dislikes_per_feature(u, user_item_binary, gender_per_ids)
#calculate odds related to gender
users_items_likes_gender = (item_likes_per_gender+1)/(((np.dot(pd.DataFrame(items_global_likes.iloc[:,1].replace(np.nan,0)),np.ones((1,item_likes_per_gender.shape[0])))).T)+2)
users_items_dislikes_gender = (item_dislikes_per_gender+1)/(((np.dot(pd.DataFrame(items_global_likes.iloc[:,0].replace(np.nan,0)),np.ones((1,item_dislikes_per_gender.shape[0])))).T)+2)
odds_gender = pd.DataFrame(users_items_likes_gender)/pd.DataFrame(users_items_dislikes_gender)

In [251]:
#Returns a dictionary of age categories with their associated users ids
age_per_ids = return_dict_feature_ids(u, ' age ', age = True)
#Return items likes and dislikes matrices per age
item_likes_per_age, item_dislikes_per_age = return_likes_dislikes_per_feature(u, user_item_binary, age_per_ids)
#calculate odds related to age
users_items_likes_age = (item_likes_per_age + 1)/(((np.dot(pd.DataFrame(items_global_likes.iloc[:,1].replace(np.nan,0)),np.ones((1,item_likes_per_age.shape[0])))).T)+2)
users_items_dislikes_age = (item_dislikes_per_age + 1)/(((np.dot(pd.DataFrame(items_global_likes.iloc[:,0].replace(np.nan,0)),np.ones((1,item_dislikes_per_age.shape[0])))).T)+2)
odds_age = pd.DataFrame(users_items_likes_age)/pd.DataFrame(users_items_dislikes_age)

## Recommandation  de 10 films sur la base de trois catégories

In [253]:
def return_recommendations(job, gender, age, odds_likes, odds_jobs, odds_gender, odds_age):
    """
    return first 10 items recommendations based on input job, gender,age
    """
    odds = odds_likes*(odds_jobs.T)[job]*(odds_gender.T)[gender]*((odds_age.T))[age]
    return  (odds/(1+odds)).sort_values(ascending=False)[0:10]

In [254]:
return_recommendations('engineer', 'F', 'young', odds_likes = odds_likes, odds_jobs = odds_jobs, odds_gender = odds_gender, odds_age = odds_age)

483    0.967840
480    0.950883
115    0.928483
83     0.927111
107    0.927056
478    0.913225
114    0.911541
302    0.911324
918    0.908075
487    0.904690
dtype: float64

## Calcul des votes et du RMSE moyen 

In [256]:
a_c = labelize_age(u)
s = pd.DataFrame(u[' gender '].values, index = range(1,N_USERS), columns = [' gender '])
j = pd.DataFrame(u[' job '].values, index = range(1,N_USERS), columns = [' job '])
new_u = pd.concat([a_c , s , j], axis = 1)

In [257]:
def calculate_votes(user_item = user_item, new_u = new_u, odds_jobs = odds_jobs, odds_gender = odds_gender, odds_age = odds_age):
    """
    calculates votes based on probabilities
    """
    #A df which contains votes
    all_votes_predictions = pd.DataFrame(np.zeros((user_item.shape[0],user_item.shape[1])), index = range(1,N_USERS) ,columns = range(1,N_ITEMS))
    #A new df with age, gender and job in the needed labelization 
    s = pd.DataFrame(u[' gender '].values, index = range(1,N_USERS), columns = [' gender '])
    j = pd.DataFrame(u[' job '].values, index = range(1,N_USERS), columns = [' job '])
    a_c = labelize_age(u)
    new_u = pd.concat([a_c , s , j], axis = 1)
    #Calculate likes and dislikes averages
    likes_avgs = user_item[user_item >= 4].mean()
    dislikes_avgs = user_item[user_item < 4].mean()
    #Loop over the new df to calculate votes
    for index, row in new_u.iterrows():
        #Calculate odd by multiplying odds matrices
        all_odds = (odds_jobs.T[row[' job ']])*(odds_gender.T[row[' gender ']])*(odds_age.T[row[' age ']])
        #Calculates probabilities based on odds
        all_probs = all_odds/(1+all_odds)
        #Calculte votes per user
        user_votes = (all_probs*likes_avgs + (1-all_probs)*dislikes_avgs)
        #Fill in the df votes
        all_votes_predictions.loc[index,:] = user_votes
    
    return all_votes_predictions

In [258]:
def compute_rmse(prediction , user_item = user_item ) : 
    ''' Return the RMSE between the prediction and the user-item matrix '''
    # Index where the user_item is not null
    indexes_not_nan = user_item.notnull()
    rmse = np.sqrt(((prediction[indexes_not_nan] - user_item[indexes_not_nan])**2).mean())
    return rmse

In [259]:
all_votes_predictions = calculate_votes(user_item, new_u, odds_jobs, odds_gender, odds_age)

In [260]:
compute_rmse(all_votes_predictions, user_item).mean()

1.021163785917258

# Question 2

In [45]:
m

,100299,100967,10151,101705,101863,102458,102886,102966,10302,103700,...,96767,97060,97150,9721,97410,97863,98185,99113,9947,9993
100299,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10151,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101705,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101863,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97863,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99113,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9947,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- Tout d'abord, pour éviter les divisions par 0, on met la diagonale de la matrice à 1

In [49]:
np.fill_diagonal(m.values, 1)

- Ensuite, nous appliquons l'algorithne PageRank

In [48]:
def page_rank(m , d = 0.85, epsilon = 1e-5, max_iter = 10000) : 
    ''' Compute the PageRank score '''
    # Number of articles 
    N = m.shape[0]
    # PageRank values
    r = np.ones((1,N))
    # Initialise the condition 
    new_r = r + 10
    # Iteration 
    iteration = 0
    # Condition : if the mean of difference is greater than epsilon
    condition = abs(r - new_r).mean() > epsilon 
    
    while condition and iteration < max_iter     : 
        # Get the number of output links 
        s = m.sum(axis=0).values.reshape(-1,1)
        # Get the new PageRank vector
        new_r = (1-d)/N +  d * m @ (r.T / s)                
        # Convert into numpy 
        new_r = new_r.values.T
        # Update the condition 
        condition = abs(r - new_r).mean() > epsilon

        r = new_r
        
        iteration+=1
        
    return pd.DataFrame(r.reshape(-1,), index = m.index, columns=['PageRank'])

In [50]:
r = page_rank(m, d= 0.85, epsilon = 1e-5, max_iter = 1000)

In [51]:
r

,PageRank
100299,0.001470
100967,0.000447
10151,0.000674
101705,0.001167
101863,0.000289
...,...
97863,0.006034
98185,0.000279
99113,0.002592
9947,0.000675


- Recommendation : prendre les articles maximum de PageRank

In [52]:
def get_recommendation(index, r = r, m=m,N= 10 ): 
    ''' Return recommendations for the index article from PageRank score '''
    # Get the element wise product between the PageRank and adjacent matrix
    idx = m[m.index == index].values * r.values.reshape(1,-1)
    idx = (-idx[0]).argsort()[:N]
    recommend = m.index[idx]
    return list(recommend)

In [53]:
get_recommendation(422908)

[422908, 155792, 3170, 131548, 241538, 17094, 124, 466838, 147460, 149673]

- Variante : etendre le sous-ensemble S aux références de références

In [54]:
m_ = m.values @ m.values + m.values.T

In [55]:
m_[m_ >1] = 1

In [56]:
new_m = m.copy()

In [57]:
m_

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [58]:
new_m[:] = m_

In [59]:
new_m

,100299,100967,10151,101705,101863,102458,102886,102966,10302,103700,...,96767,97060,97150,9721,97410,97863,98185,99113,9947,9993
100299,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100967,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10151,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101705,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101863,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97863,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
98185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
99113,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9947,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [60]:
new_r = page_rank(new_m, d= 0.85, epsilon = 1e-5, max_iter = 1000)

In [61]:
def get_recommendation_new(index, r = new_r, m=new_m,N= 10 ): 
    ''' Return recommendations for the index article from PageRank score '''
    # Get the element wise product between the PageRank and adjacent matrix
    idx = m[m.index == index].values * r.values.reshape(1,-1)
    idx = (-idx[0]).argsort()[:N]
    recommend = m.index[idx]
    return list(recommend)

In [62]:
get_recommendation_new(422908)

[311874, 19422, 422908, 83263, 464834, 297641, 366858, 177173, 17507, 226359]

# Question 3

- On considère pour cette question qu'un utilisateur est représenté par son vecteur dans la matrice d'adjacence
- On calcule ensuite la distance cosinus pour chaque utilisateur 
- La première recommendation sera forcément l'utilisateur lui-même 

In [64]:
def get_recommendation_sim(m = m , N=10) :
    ''' Return the top N recommendation using the cosine similarity '''
    # Get the similarity for each user 
    df_sim = pd.DataFrame(cosine_similarity(m,m), index = m.index, columns = m.columns)
    # Get the indexes that maximises the cosine similarity
    idx = (-df_sim.values).argsort()
    # Keep the N max
    idx = idx[:,:N]
    # Convert the indexes into article indexes
    article_idx = np.zeros(idx.shape)
    for i,index in enumerate(idx) : 
        for j,index_ in enumerate(index) : 
            article_idx[i,j] = m.index[index_]
    return pd.DataFrame(article_idx, index = m.index), df_sim

In [65]:
df_sim, df_sim_PageRank = get_recommendation_sim(N=10)

In [66]:
df_sim

,0,1,2,3,4,5,6,7,8,9
100299,100299.0,527057.0,170577.0,66991.0,25937.0,164141.0,93456.0,27921.0,120053.0,5101.0
100967,100967.0,281158.0,282066.0,517603.0,20858.0,549091.0,185918.0,275211.0,298007.0,272596.0
10151,10151.0,45228.0,93250.0,130506.0,189479.0,394586.0,3950.0,395364.0,40057.0,396568.0
101705,101705.0,74362.0,42779.0,114285.0,88318.0,402497.0,40057.0,40014.0,40009.0,399339.0
101863,101863.0,77030.0,134503.0,223844.0,224357.0,66776.0,145363.0,258875.0,135666.0,125089.0
...,...,...,...,...,...,...,...,...,...,...
97863,97863.0,289801.0,171813.0,295968.0,170200.0,503616.0,342910.0,520768.0,284612.0,8069.0
98185,98185.0,136975.0,124.0,83730.0,147460.0,395364.0,17507.0,1347.0,422908.0,373307.0
99113,99113.0,150689.0,226964.0,76418.0,236799.0,248936.0,3950.0,395364.0,396568.0,403583.0
9947,9947.0,31170.0,77621.0,139321.0,16117.0,97150.0,50568.0,79248.0,42422.0,340170.0


- 10 meilleures recommendations de 422908

In [67]:
df_sim[df_sim.index == 422908]

,0,1,2,3,4,5,6,7,8,9
422908,422908.0,497542.0,96767.0,147460.0,466838.0,149673.0,155792.0,70445.0,124.0,3170.0


# Question 4

In [68]:
abstract = pd.read_csv(ABSTRACT_PATH)

In [69]:
abstract

,Unnamed: 0,Id,Titre,Auteurs,Description
0,1,124,Hybrid Automata: An Algorithmic Approach to th...,Costas Courcoubetis; Pei-hsin Ho; Rajeev Alur;...,We introduce the framework of hybrid automata ...
1,2,496,Representing Action: Indeterminacy and Ramific...,Enrico Giunchiglia; G. Neelakantan Kartha; Vla...,We define and study a high-level language for ...
2,3,712,A Decision Procedure for a Temporal Belief Logic,Michael Fisher; Michael Wooldridge;,. This paper presents a temporal belief logic ...
3,4,1186,A Comparative Evaluation of Sequential Feature...,David W. Aha; Richard L. Bankert;,Several recent machine learning publications d...
4,5,1347,Extending Promela and Spin for Real Time,Costas Courcoubetis; Stavros Tripakis;,The efficient representation and manipulation ...
...,...,...,...,...,...
1085,1086,548670,Null,NaN,NaN
1086,1087,549091,Null,NaN,NaN
1087,1088,561317,Null,NaN,NaN
1088,1089,570764,Null,NaN,NaN


- On ne considère que les descriptions (peut être à modifier)

In [26]:
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/ghaith/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ghaith/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
def get_vocab_and_lemmatize(sentences = abstract['Description']) : 
    ''' Return the vocabulary of the sentences, and the lemmatisated sentences  '''
    vocab = set()
    new_sentences = []
    lemmatizer = WordNetLemmatizer() 

    for sentence in sentences : 
        n_sentence = []
        try : 
            for word in  word_tokenize(sentence) : 
                lem_word = lemmatizer.lemmatize(word)
                n_sentence.append(lem_word)
                vocab.add(lem_word)
        except : 
            pass
        
        new_sentences.append(n_sentence)
    return vocab, new_sentences

In [70]:
vocab, new_sentences = get_vocab_and_lemmatize()

In [29]:
def get_df(vocab = vocab, new_sentences = new_sentences) : 
    ''' Return the df : number of documents that contain the given word '''
    df_vocab = { x : 0 for x in list(vocab)}
    # Loop over the vocabulary
    for word in df_vocab : 
        # Loop over the sentences
        for sentence in new_sentences :
            # If the word is in the document 
            if word in sentence : 
                # Increment the counter
                df_vocab[word]+=1
                # Pass to the next document
                break
    return df_vocab

In [71]:
df_vocab = get_df()

In [31]:
def get_term_document_matrix(vocab = vocab, new_sentences = new_sentences, abstract = abstract, df_vocab = df_vocab) : 
    ''' Return the term document matrix with TF-IDF '''
    # Size of the vocabulary
    n_vocab = len(vocab)
    # Number of documents 
    n_document = abstract.shape[0]
    # Frequency matrix 
    tf_matrix = pd.DataFrame(np.zeros((n_vocab, n_document)), index = vocab, columns = abstract.index)
    # Df matrix 
    df_matrix = tf_matrix.copy()
    # Loop over the sentences 
    for i,sentence in enumerate(new_sentences) : 
        # Loop over the words 
        for word in sentence :
            # Get the index of the word in the matrix 
            i_word = (tf_matrix.index == word)
            # Compute the frequency 
            tf_matrix.iloc[i_word, i] += 1
            # Compute the inverse frequency 
            df_matrix.iloc[i_word,i] = np.log(n_document / df_vocab[word])
        # Normalise the frequency 
        tf_matrix.iloc[:,i]/=len(sentence)
    
    matrix = df_matrix * tf_matrix
    return matrix.fillna(0)
# return  df_matrix, tf_matrix

In [72]:
term_document = get_term_document_matrix()

In [75]:
term_document.iloc[1][term_document.iloc[1]>0]

480    0.041631
Name: undergraduate, dtype: float64

In [79]:
def convert_columns(matrix, abstract = abstract) : 
    ''' Convert the column to the following id '''
    new_matrix = matrix.copy()
    new_matrix.columns = abstract['Id'].values
    return new_matrix

In [80]:
# Convert the columns name 
term_document = convert_columns(term_document)

In [81]:
term_document

,124,496,712,1186,1347,1948,2104,2111,2490,3156,...,543817,545708,547939,548310,548620,548670,549091,561317,570764,573595
happened,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
undergraduate,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
inadequate,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
apportioning,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Surface,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PODC,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fChristian.Frei,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
experimental,0.0,0.0,0.0,0.0,0.0,0.0,0.0409,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TACAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
def get_recommendation_term_document(m = term_document , N=10) :
    ''' Return the top N recommendation using the cosine similarity '''
    # Get the similarity for each user 
    df_sim = pd.DataFrame(cosine_similarity(m.T,m.T), index = m.columns, columns = m.columns)
    # Get the indexes that maximises the cosine similarity
    idx = (-df_sim.values).argsort()
    # Keep the N max
    idx = idx[:,:N]
    # Convert the indexes into article indexes
    article_idx = np.zeros(idx.shape)
    for i,index in enumerate(idx) : 
        for j,index_ in enumerate(index) : 
            article_idx[i,j] = m.columns[index_]
    return pd.DataFrame(article_idx, index = m.columns), df_sim

In [85]:
df_sim_term_document = get_recommendation_term_document(m =term_document)

- 10 recommendations de l'article 422908

In [86]:
df_sim_term_document[df_sim_term_document.index == 422908]

,0,1,2,3,4,5,6,7,8,9
124,124.0,53595.0,66291.0,61863.0,247211.0,322240.0,10563.0,17507.0,97410.0,32269.0
496,496.0,3177.0,298240.0,434053.0,9181.0,226801.0,86700.0,89365.0,157215.0,64835.0
712,712.0,67682.0,453924.0,179072.0,54955.0,437951.0,126336.0,105199.0,131548.0,87538.0
1186,1186.0,184608.0,122844.0,87538.0,138988.0,3156.0,141506.0,145880.0,109288.0,135793.0
1347,1347.0,220463.0,9181.0,95136.0,345650.0,83730.0,53595.0,397021.0,50929.0,99113.0
...,...,...,...,...,...,...,...,...,...,...
548670,124.0,248596.0,248598.0,248906.0,248936.0,249024.0,249127.0,250620.0,251082.0,251381.0
549091,124.0,248596.0,248598.0,248906.0,248936.0,249024.0,249127.0,250620.0,251082.0,251381.0
561317,124.0,248596.0,248598.0,248906.0,248936.0,249024.0,249127.0,250620.0,251082.0,251381.0
570764,124.0,248596.0,248598.0,248906.0,248936.0,249024.0,249127.0,250620.0,251082.0,251381.0


# Question 5

In [28]:
# TO DO

In [226]:
df_sim_term_document, df_sim_tfidf = get_recommendation_term_document(m =term_document)
df_new_sim, df_new_sim_PageRank = get_recommendation_sim(new_m, N=10)

In [125]:
m_base = pd.read_table(TABLE_PATH, sep=' ')
def return_df_citations(m_base = m_base):  
    """
    returns a 2 columns df with refree/referenced papers
    """
    citations = []
    for i, (index, row) in enumerate(m_base.iterrows()):
        indexes = row[row ==1].index
        for k in indexes:
            citations.append([index,k])

    return pd.DataFrame(citations, columns = ['referee','referenced'])

In [225]:
df_citations = return_df_citations(m_base = m_base)

In [217]:
# Copy the original citations matrix (not the one where we have ones in the diagonal)
m_copy = m_base.copy()
# Copy the tfidf simlarities matrix
df_sim_tfidf_copy = df_sim_tfidf.copy()

def calculate_avg_similarities(df_sim_tfidf_copy, df_citations, train_index):
    """
    calcule average similarities between cited papers only on train datatset
    """
    avg_sim = 0
    for idx,row in df_citations.iloc[train_index].iterrows():
        avg_sim +=  df_sim_tfidf_copy.loc[int(df_citations.iloc[idx][0]), int(df_citations.iloc[idx][1])]/len(df_citations)  
    return avg_sim

def get_predictions_tfidf():
    """
    predict links based on tfidf similarities
    """
    count_predicted_links = 0
    # Get the indexes of the test 
    indexes_test = df_citations.iloc[test_index]
    # Convert the test rating to NaN
    for j,k in zip(indexes_test['referee'].values, indexes_test['referenced'].values) :            
        m_copy.loc[j,k] = np.nan
        df_sim_tfidf_copy.loc[j,k] = np.nan
        
    #Return average similarities on train dataset
    avg_sim = calculate_avg_similarities(df_sim_tfidf_copy, df_citations, train_index)
    
    #calculate similarities for missed values based on tfidf vectors
    for idx,row in df_citations.iloc[test_index].iterrows():
        predicted_sim = df_sim_tfidf_copy.loc[int(df_citations.iloc[idx][0]), int(df_citations.iloc[idx][1])]  
        #We fill missed values with ones if the similarity is higher than the average similarity between liked papers
        if predicted_sim >= avg_sim:
            m_copy.loc[df_citations.iloc[idx][0], df_citations.iloc[idx][1]] = 1
            count_predicted_links += 1
        else:
            m_copy.loc[df_citations.iloc[idx][0], df_citations.iloc[idx][1]] = 0 
    #A rectified recall to measure performance by calculating truly predicted ones(links) among the true ones  
    rect_recall = count_predicted_links/len(test_index)
    return rect_recall

In [224]:
def avg_recall():
    """
    return an average rectified recall to measure performance by calculating truly 
    predicted ones(links) among the true ones for all folds 
    """
    n_splits=10
    kf = KFold(n_splits=10, random_state=None, shuffle=False)
    avg_recall = 0
    for train_index, test_index in kf.split(df_citations):
        avg_recall += get_predictions_tfidf()/n_splits
    return avg_recall
    

0.7775747308727412
